# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [53]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,f1_score
from sklearn.multioutput import MultiOutputClassifier
import pickle
from xgboost import XGBClassifier

[nltk_data] Downloading package punkt to /home/neil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/neil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/neil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_messages.db')


df = pd.read_sql('SELECT * FROM message_category',engine)
X = df.message
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    """
    tokenize text by:
    1. Replace any urls with urlplaceholder
    2. Split into tokens (words)
    3. lemmatize
    
    Args:
        text:
            the text to tokenize.
    """
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('vect',  CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',   RandomForestClassifier())
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f7bdc0c03a0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
def evaluate_model(model, X_test, y_test, category_names):
    """
    Calculate and print overall accuracy and f1 score followed by a classification report showing metric for each category
    
            
    """
    y_pred = model.predict(X_test)
    
    accuracy = (y_test == y_pred).mean().mean()
    f1_samples = f1_score(y_test, y_pred, average='samples')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    
    print(f'''
    Overall accuracy      : {accuracy}
    Overall f1 [samples]  : {f1_samples}
    Overall f1 [weighted] : {f1_weighted}
    ''')
        
    labels = [1,0]
    n=0

    for category_name in category_names:

        print(classification_report(y_test.iloc[:,n],y_pred[:,n],labels=labels,target_names=[category_name + '-' + str(v) for v in labels]))

        n+=1

In [21]:
evaluate_model(pipeline,X_test,y_test,y_test.columns)

/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defi


    Overall accuracy      : 0.9415327365815619
    Overall f1 [samples]  : 0.4775505621072902
    Overall f1 [weighted] : 0.4660482909194558
    
              precision    recall  f1-score   support

   related-1       0.83      0.96      0.89      5042
   related-0       0.73      0.33      0.45      1512

    accuracy                           0.82      6554
   macro avg       0.78      0.64      0.67      6554
weighted avg       0.80      0.82      0.79      6554

              precision    recall  f1-score   support

   request-1       0.92      0.42      0.58      1102
   request-0       0.89      0.99      0.94      5452

    accuracy                           0.90      6554
   macro avg       0.91      0.71      0.76      6554
weighted avg       0.90      0.90      0.88      6554

              precision    recall  f1-score   support

     offer-1       0.00      0.00      0.00        31
     offer-0       1.00      1.00      1.00      6523

    accuracy                       

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fc1e6f38160>)),
  ('tfidf', TfidfTransformer()),
  ('clf', RandomForestClassifier())],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fc1e6f38160>),
 'tfidf': TfidfTransformer(),
 'clf': RandomForestClassifier(),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__bootstrap': True,
 'clf__ccp_alpha': 0.0,


In [22]:
param_grid = {
    'clf__max_features': ['sqrt', 'log2'],
    'clf__n_estimators': [100, 200]
}

grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid, 
                          cv = 3, n_jobs = 3, verbose = 2)

In [23]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:  4.8min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f7bdc0c03a0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=3,
             param_grid={'clf__max_features': ['sqrt', 'log2'],
                         'clf__n_estimators': [100, 200]},
             verbose=2)

In [24]:
grid_search.best_params_

{'clf__max_features': 'sqrt', 'clf__n_estimators': 200}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
evaluate_model(grid_search.best_estimator_,X_test,y_test,y_test.columns)

/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defi


    Overall accuracy      : 0.9417785576238429
    Overall f1 [samples]  : 0.47827975509606613
    Overall f1 [weighted] : 0.4681103840202724
    
              precision    recall  f1-score   support

   related-1       0.83      0.96      0.89      5042
   related-0       0.72      0.33      0.46      1512

    accuracy                           0.82      6554
   macro avg       0.77      0.65      0.67      6554
weighted avg       0.80      0.82      0.79      6554

              precision    recall  f1-score   support

   request-1       0.90      0.42      0.57      1102
   request-0       0.89      0.99      0.94      5452

    accuracy                           0.89      6554
   macro avg       0.90      0.70      0.76      6554
weighted avg       0.89      0.89      0.88      6554

              precision    recall  f1-score   support

     offer-1       0.00      0.00      0.00        31
     offer-0       1.00      1.00      1.00      6523

    accuracy                      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [30]:
pipeline_with_starting_verb = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier(max_features='sqrt',n_estimators=200))
    ])

In [31]:
pipeline_with_starting_verb.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f7bdc0c03a0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf',
                 RandomForestClassifier(max_features='sqrt',
                                        n_estimators=200))])

In [32]:
evaluate_model(pipeline_with_starting_verb,X_test,y_test,y_test.columns)

/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defi


    Overall accuracy      : 0.9418124639745025
    Overall f1 [samples]  : 0.47836311549558064
    Overall f1 [weighted] : 0.4688669841018403
    
              precision    recall  f1-score   support

   related-1       0.83      0.96      0.89      5042
   related-0       0.73      0.33      0.46      1512

    accuracy                           0.82      6554
   macro avg       0.78      0.65      0.67      6554
weighted avg       0.80      0.82      0.79      6554

              precision    recall  f1-score   support

   request-1       0.92      0.42      0.58      1102
   request-0       0.90      0.99      0.94      5452

    accuracy                           0.90      6554
   macro avg       0.91      0.71      0.76      6554
weighted avg       0.90      0.90      0.88      6554

              precision    recall  f1-score   support

     offer-1       0.00      0.00      0.00        31
     offer-0       1.00      1.00      1.00      6523

    accuracy                      

In [69]:
xgboost_with_starting_verb = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(xgb.XGBClassifier(objective='binary:logistic')))
    ])

In [70]:
xgboost_with_starting_verb.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f7bdc0c03a0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsa...
                                                 

In [72]:
evaluate_model(xgboost_with_starting_verb,X_test,y_test,y_test.columns)

/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defi


    Overall accuracy      : 0.9507255959041129
    Overall f1 [samples]  : 0.5266662971910298
    Overall f1 [weighted] : 0.6465274151547071
    
              precision    recall  f1-score   support

   related-1       0.84      0.94      0.89      5042
   related-0       0.67      0.42      0.51      1512

    accuracy                           0.82      6554
   macro avg       0.76      0.68      0.70      6554
weighted avg       0.80      0.82      0.80      6554

              precision    recall  f1-score   support

   request-1       0.80      0.57      0.67      1102
   request-0       0.92      0.97      0.94      5452

    accuracy                           0.90      6554
   macro avg       0.86      0.77      0.81      6554
weighted avg       0.90      0.90      0.90      6554

              precision    recall  f1-score   support

     offer-1       0.00      0.00      0.00        31
     offer-0       1.00      1.00      1.00      6523

    accuracy                       

In [76]:
param_grid = param_test1 = {'clf__estimator__max_depth':range(3,10,2), 'clf__estimator__min_child_weight':range(1,6,2)}


gsearch_xgboost_with_starting_verb = GridSearchCV(estimator = xgboost_with_starting_verb,
                        param_grid = param_grid,
                        n_jobs=3, iid=False, 
                        cv=3)

In [77]:
gsearch_xgboost_with_starting_verb.fit(X_test,y_test)

/home/neil/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f7bdc0c03a0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                    

In [78]:
evaluate_model(gsearch_xgboost_with_starting_verb.best_estimator_,X_test,y_test,y_test.columns)

/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/neil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defi


    Overall accuracy      : 0.9655596243176348
    Overall f1 [samples]  : 0.6089156972868318
    Overall f1 [weighted] : 0.7591342689720009
    
              precision    recall  f1-score   support

   related-1       0.85      0.97      0.91      5042
   related-0       0.81      0.44      0.57      1512

    accuracy                           0.85      6554
   macro avg       0.83      0.70      0.74      6554
weighted avg       0.84      0.85      0.83      6554

              precision    recall  f1-score   support

   request-1       0.93      0.66      0.77      1102
   request-0       0.94      0.99      0.96      5452

    accuracy                           0.93      6554
   macro avg       0.93      0.82      0.87      6554
weighted avg       0.93      0.93      0.93      6554

              precision    recall  f1-score   support

     offer-1       1.00      0.52      0.68        31
     offer-0       1.00      1.00      1.00      6523

    accuracy                       

### 9. Export your model as a pickle file

In [79]:
pickle.dump(gsearch_xgboost_with_starting_verb, open('xgboost_with_starting_verb.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.